# Predictive Models

In the last two notebooks we had a look at two of the components of the Basis Mixer. In this notebook we add the third part of the puzzle: the **Predictive Models**.

A predictive model is defined as a mathematical which maps score information (encoded by the basis functions) $\mathbf{\Phi}$ to expressive parameters $\mathbf{Y}$

$$F(\boldsymbol{\Phi}) = \mathbf{Y}$$

In [1]:
import json
import logging
import os

import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

from basismixer.predictive_models import (construct_model,
                                          RecurrentModel,
                                          SupervisedTrainer,
                                          MSELoss)
from basismixer.utils import load_pyc_bz, save_pyc_bz

from helper.predictive_models_helper import construct_dataloaders

logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)



The dataset that we created previously

In [5]:
dataset_fn = ['../vienna4x22_notewise.pyc.bz', '../vienna4x22_onsetwise.pyc.bz']


We define the configuration of the model

In [6]:
model_config = [
    dict(input_type='notewise',
         basis_functions='../vienna4x22_notewise.pyc.bz',
         parameter_names=['velocity_trend', 'timing', 'log_articulation'],
         model=dict(constructor=['basismixer.predictive_models', 'FeedForwardModel'],
         args=dict(
             hidden_size=128,
         )),
         train_args=dict(
             lr=1e-4,
             epochs=3,
             save_freq=1,
             early_stopping=100,
             batch_size=10,
         )
         ),
    dict(input_type='onsetwise',
         basis_functions='../vienna4x22_onsetwise.pyc.bz',
         parameter_names=['velocity_trend', 'beat_period_standardized'],
         constructor=['basismixer.predictive_models', 'RecurrentModel'],
         args=dict(
             recurrent_size=128,
             n_layers=1,
             hidden_size=64),
         train_args=dict(
             lr=1e-4
             epochs=3,
             save_freq=1,
             early_stopping=100,
             batch_size=10,
         )
         )
]



## Training the models

Given a training set of expressive performances aligned to their scores, we can train the models in a supervised way by minimizing the *mean squared error* between predictions and the observed expressive parameters.

In [ ]:
rng = np.random.RandomState(1984)

datasets = []
models = []
target_idxs = []
input_idxs = []
valid_size = 0.20
for fn, config in zip(dataset_fn, model_config):

    #### Construct Dataset ####
    dataset, train_dataloader, valid_dataloader = construct_dataset(dataset_fn, 
                                                                    batch_size=config['train_args'].pop('batch_size'), 
                                                                    valid_size=0.2)

    datasets.append((dataset, train_dataloader, valid_dataloader))

    #### Construct Models ####

    if config['basis_functions'] == 'from_dataset':
        model_in_names = in_names
    elif isinstance(config['basis_functions'], (list, tuple)):
        model_in_names = []
        for bf in config['basis_functions']:
            for name in in_names:
                if name.startswith(bf):
                    model_in_names.append(name)

    i_idxs = np.array([list(in_names).index(bf) for bf in model_in_names])
    input_idxs.append(i_idxs)

    model_out_names = []
    for pn in config['parameter_names']:
        for name in out_names:
            if name == pn:
                model_out_names.append(name)

    t_idxs = np.array([list(out_names).index(pn) for pn in model_out_names])
    target_idxs.append(t_idxs)

    config['args']['input_names'] = model_in_names
    config['args']['input_size'] = len(model_in_names)
    confit['args']['input_type'] = config['input_type']

    config['args']['output_names'] = model_out_names
    config['args']['output_size'] = len(model_out_names)

    model = construct_model(config)
    models.append(model)




In [ ]:
def train_model(model, config, train_loader, val_loader, out_dir):
    # Get the configuration for the trainer
    t_config = config['train_args']

    # Name of the model
    model_name = '-'.join(model.output_names) + '-' + model.input_type
    # Create a directory for storing the model parameters
    model_out_dir = os.path.join(out_dir, model_name)
    if not os.path.exists(model_out_dir):
        os.mkdir(model_out_dir)
    # Loss function
    loss = MSELoss()
    # Initialize the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=t_config.pop('lr'))
    # Create trainer for training model in a supervised way
    trainer = SupervisedTrainer(model=model,
                                train_loss=loss,
                                optimizer=optimizer,
                                valid_loss=loss,
                                train_dataloader=train_loader,
                                valid_dataloader=val_loader,
                                out_dir=model_out_dir,
                                **t_config)
    # train the mode
    trainer.train()

In [ ]:
for (model, dtv, config, t_idxs) in zip(models, datasets,
                            model_config, target_idxs):
    dataset, train_loader, val_loader = dtv
    # set the indices of the desired targets in the dataset
    for d in dataset.datasets:
        d.targets_idx = t_idxs
    # Train the models
    train_model(model, config, train_loader, val_loader, out_dir)

In [ ]:
plot_predictions(models, targets)